In [1]:
url="https://s3.us-east-2.amazonaws.com/datafaculty/bootcamp/HR_comma_sep.csv"

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
data=pd.read_csv(url)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [6]:
data.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
sales                    0
salary                   0
dtype: int64

In [7]:
data=pd.get_dummies(data)

In [8]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [9]:
### create train and test parts of the data
import sklearn.model_selection as model_selection

In [10]:
y=data['left']
x=data.drop('left',axis=1)

In [11]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(x,
                                                               y,
                                                    test_size=0.2,
                                                    random_state=200)

In [12]:
## Model 
import sklearn.ensemble as ensemble

C:\Users\Home\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [14]:
clf=ensemble.RandomForestClassifier(n_estimators=100,oob_score=True,n_jobs=-1)

In [15]:
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [38]:
pred_y=clf.predict(x_test)

In [20]:
clf.predict_proba(x_test)[:,1]

array([0.  , 0.  , 0.01, ..., 0.  , 0.19, 1.  ])

In [21]:
### Variable importance
pd.Series(clf.feature_importances_,index=x_train.columns).sort_values(ascending=False)

satisfaction_level       0.306314
time_spend_company       0.187638
number_project           0.176368
average_montly_hours     0.158139
last_evaluation          0.122830
Work_accident            0.010263
salary_low               0.006724
salary_high              0.005452
sales_technical          0.003951
sales_sales              0.003543
salary_medium            0.003400
sales_support            0.002997
sales_hr                 0.001829
sales_IT                 0.001731
sales_RandD              0.001695
promotion_last_5years    0.001664
sales_accounting         0.001638
sales_management         0.001423
sales_marketing          0.001293
sales_product_mng        0.001107
dtype: float64

In [23]:
clf.oob_score_

0.9908325693807817

In [24]:
### If 100, 200, or 300 tree is better
for i in [100,200,300]:
    clf=ensemble.RandomForestClassifier(n_estimators=i,
                                        oob_score=True,
                                        n_jobs=-1)
    clf.fit(x_train,y_train)
    print("oob score for {} tree is {}".format(i,clf.oob_score_))

oob score for 100 tree is 0.9898324860405033
oob score for 200 tree is 0.9906658888240687
oob score for 300 tree is 0.990415867988999


In [32]:
### get prob predictions on oob data
pred_oob=clf.oob_decision_function_[:,1]

In [33]:
import sklearn.metrics as metrics

In [34]:
metrics.roc_auc_score(y_train,pred_oob)

0.993777600422144

In [35]:
### If 100, 200, or 300 tree is better
for i in [100,200,300]:
    clf=ensemble.RandomForestClassifier(n_estimators=i,
                                        oob_score=True,
                                        n_jobs=-1)
    clf.fit(x_train,y_train)
    pred_oob=clf.oob_decision_function_[:,1]
    auc=metrics.roc_auc_score(y_train,pred_oob)
    print("Auc for {} tree is {}".format(i,auc))

Auc for 100 tree is 0.9921927994920152
Auc for 200 tree is 0.9927182536211026
Auc for 300 tree is 0.993975015874525


In [39]:
results=pd.DataFrame({'actual':y_test,'predicted':pred_y})

In [41]:
results.head()

,actual,predicted
11291,0,0
6315,0,0
9333,0,0
5519,0,0
7816,0,0
